In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# 假设 filtered_data 和 ST_data 已经定义并经过预处理

# 过滤并按天重采样
filtered_data = filtered_data[filtered_data['albedo_Avg'] >= 0.1]
ST_data = ST_data[ST_data >= -40]

# 按天重采样计算平均值
daily_mean_albedo = filtered_data['albedo_Avg'].resample('D').mean()
daily_mean_TC = ST_data.resample('D').mean()

# 获取模型数据
albedo_mod = closest_data_cosipy['ALBEDO'].to_dataframe()
ST_mod = closest_data_cosipy['TS'].to_dataframe()

# 对模型数据按天重采样并计算平均值
albedo_mod_mean = albedo_mod['ALBEDO'].resample('D').mean()
ST_mod_mean = ST_mod['TS'].resample('D').mean()

# 三天滑动平均处理
daily_mean_albedo_smooth = daily_mean_albedo.rolling(window=3).mean()
albedo_mod_mean_smooth = albedo_mod_mean.rolling(window=3).mean()

daily_mean_TC_smooth = daily_mean_TC.rolling(window=3).mean()  # 三天滑动平均处理 TC 数据
ST_mod_mean_smooth = ST_mod_mean.rolling(window=3).mean()  # 三天滑动平均处理温度数据

# 1. 处理 Albedo 数据
true_values_albedo = daily_mean_albedo_smooth
predicted_values_albedo = albedo_mod_mean_smooth

# 合并 albedo 数据的真实值和预测值
merged_albedo = pd.merge(
    true_values_albedo, predicted_values_albedo,
    how="inner",  # 仅保留两者都有的时间戳
    left_index=True, right_index=True
)

# 删除含有 NaN 的行
merged_albedo = merged_albedo.dropna()

# 计算 RMSE 和 R2 for albedo 数据
rmse_albedo = np.sqrt(mean_squared_error(merged_albedo['albedo_Avg'], merged_albedo['ALBEDO']))
r2_albedo = r2_score(merged_albedo['albedo_Avg'], merged_albedo['ALBEDO'])

# 打印 albedo 数据的 RMSE 和 R2
print(f"Albedo RMSE: {rmse_albedo:.4f}")
print(f"Albedo R2: {r2_albedo:.4f}")

# 2. 处理温度数据 TS
true_values_TC = daily_mean_TC_smooth
predicted_values_TC = ST_mod_mean_smooth

# 合并温度数据的真实值和预测值
merged_TC = pd.merge(
    true_values_TC, predicted_values_TC,
    how="inner",  # 仅保留两者都有的时间戳
    left_index=True, right_index=True
)

# 删除含有 NaN 的行
merged_TC = merged_TC.dropna()

# 计算 RMSE 和 R2 for 温度数据
rmse_TC = np.sqrt(mean_squared_error(merged_TC['TS'], merged_TC['TS']))
r2_TC = r2_score(merged_TC['TS'], merged_TC['TS'])

# 打印温度数据的 RMSE 和 R2
print(f"Temperature RMSE: {rmse_TC:.4f}")
print(f"Temperature R2: {r2_TC:.4f}")


In [ ]:
# 合并两个 Series
merged = pd.merge(
    true_values, predicted_values,
    how="inner",  # 仅保留两者都有的时间戳
    left_index=True, right_index=True
)

# 删除含有 NaN 的行
merged_cleaned = merged.dropna()

# 提取对齐后的值
true_aligned = merged_cleaned["true_values"].values
predicted_aligned = merged_cleaned["predicted_values"].values

# 计算 RMSE 和 R2
rmse = np.sqrt(mean_squared_error(true_aligned, predicted_aligned))
r2 = r2_score(true_aligned, predicted_aligned)

print(f"RMSE: {rmse:.4f}")
print(f"R2: {r2:.4f}")


In [ ]:
import pandas as pd

# 加载数据
path = "/mnt/c/Users/78631/Desktop/冰面气象站20200916-今.csv"
data = pd.read_csv(path)

# 打印列名检查是否包含 'albedo_Avg' 和 'TargetTC_0cm_Avg'
print(data.columns)

# 将除 'TIMESTAMP' 列外的所有列转换为数值类型，忽略无法转换的内容并将其转换为 NaN
data.iloc[:, 1:] = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# 确保索引是时间格式
data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'])  # 转换为 datetime
data.set_index('TIMESTAMP', inplace=True)  # 将 'TIMESTAMP' 设置为索引

# 筛选 albedo_Avg > 0 的数据，并确保没有 NaN 值
filtered_data = data.dropna(subset=['albedo_Avg'])  # 去除 'albedo_Avg' 列中为 NaN 的行
filtered_data = filtered_data[filtered_data['albedo_Avg'] > 0]  # 筛选出 albedo_Avg > 0 的数据

# 获取目标温度数据（TargetTC_0cm_Avg）
ST_data = filtered_data['TargetTC_0cm_Avg']

# 输出一下 ST_data 以确认数据
print(ST_data)


In [ ]:
# 绘制第二个数据集的曲线（应用滑动平均）- TS (Temperature)
plt.subplot(2, 1, 2)  # 创建第二个子图

# 绘制 'Modelled' 曲线
plt.plot(ST_mod_mean_smooth.index, ST_mod_mean_smooth.values-273.15, 
         markerfacecolor='none',  # 空心圆圈
         markeredgewidth=3,
         linewidth=3,
         linestyle='-', 
         color="#373A40",         # 黑色
         label='Modelled', alpha=0.9)

# 绘制 'Measurement' 曲线，这样它会被 'Modelled' 曲线盖住
plt.plot(daily_mean_TC_smooth.index, daily_mean_TC_smooth.values, 
         markerfacecolor='none',  # 空心圆圈
         markeredgewidth=3,
         linewidth=3,       # 圆圈边缘宽度
         linestyle='-', 
         color="#CC2B52",   # 红色
         label='Measurement', alpha=0.7)
